# Диабет -- построение модели 

In [1]:
import pandas as pd
import numpy as np

from keras import models
from keras import layers
from keras import optimizers 
from keras import metrics

import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
num_data = pd.read_csv("data.csv")

In [3]:
input_size = len(num_data.columns) - 1

In [4]:
# for smart weight initialization
pos = num_data.diabetes.mean()
neg = 1 - pos

Разбиение на train и test (с стратификацией)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
def sample(data, cut, target, to_stratify = False):
    assert(cut) >= 0
    return data.groupby(target, group_keys=False).apply(lambda x: x.sample(frac=1/cut)) if to_stratify else data
    

In [7]:
def get_train_test_split(data, target, cut, iteration_num, to_stratify = True):
    r_state = 1926 * (iteration_num + 31)
    num_data_new = sample(data, cut, target, to_stratify=to_stratify)
    
    train, test = train_test_split(num_data_new, test_size=0.2, random_state=r_state * iteration_num) 

    y_train = np.array(train.diabetes)
    y_test = np.array(test.diabetes)

    x_train = np.array(train.drop(columns=[target]))
    x_test = np.array(test.drop(columns=[target]))
    
    return x_train, x_test, y_train, y_test 

## Создаём модель сети

In [8]:
params = {"first" : 10, "second" : 10, "output" : 1}

In [9]:
# для создания модели
model = models.Sequential()

initial_bias = np.log([pos / neg])
output_bias = tf.keras.initializers.Constant([initial_bias])

model.add(layers.Input(shape=(input_size,)))
model.add(layers.Dense(params["first"], activation='relu'))
model.add(layers.Dense(params["second"], activation='relu'))
model.add(layers.Dense(params["output"], activation='sigmoid', bias_initializer=output_bias))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                160       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 281
Trainable params: 281
Non-trainable params: 0
_________________________________________________________________


In [11]:
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_precision', 
#     verbose=1,
#     patience=10,
#     mode='max',
#     restore_best_weights=True)

Пытаемся учесть дисбаланс данных -- диагноз диабета редкий (10% данных).

In [12]:
weight_for_0 = (1 / neg) * 10
weight_for_1 = (1 / pos) * 10

class_weight = {0 : weight_for_0, 1 : weight_for_1}

print(weight_for_0, weight_for_1)

10.928961748633878 117.6470588235294


In [13]:
# METRICS = [metrics.Precision(name="precision"), metrics.BinaryAccuracy(name="accuracy")]

In [14]:
# model file name
modelDir =lambda x: "./models/{}models/".format(x)

def get_model_name(cut, params):
    return "cut_{}_{}l1_{}l2_{}l3".format(cut, params["first"], params["second"], params["output"])

In [20]:
## this function is task-specific
def model_train(model, split, epochs, random_target=False):
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              ) # metrics=METRICS
    
    x_train, x_test, y_train, y_test = split

    if random_target:
        y_train = np.random.randint(2, size = len(y_train))
        y_test = np.random.randint(2, size = len(y_test))  

    # batch_size made bigger to ensure some important cases are included    
    history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=min(512, len(x_train)),  
                    validation_data=(x_test, y_test), 
                    class_weight=class_weight)
    # callbacks=[early_stopping]
    return history, model

In [21]:
volume = len(num_data)
# for logarithmic scale (to divide the data uniformly)
# log_cuts = list(map(lambda x: round(x, 2), np.logspace(0, np.log(volume) - 2, base=2, num=20))) 

data_vol = [volume, 2500, 100, 60, 30, 15]
log_cuts = list(map(lambda x: int(volume / x), data_vol))

In [22]:
volume

100000

In [23]:
print("Cuts are {}".format(log_cuts))
print("Volume of data is {}".format(list(map(lambda x: int(volume / x), log_cuts))))
print(len(log_cuts))

Cuts are [1, 40, 1000, 1666, 3333, 6666]
Volume of data is [100000, 2500, 100, 60, 30, 15]
6


In [24]:
tries = 30
histories = [[] for _ in range(tries)]

# before compiling
initial_model = model
to_stratify = True
series = [1]
for i in series:
    for j in range(len(log_cuts)):
        cut = log_cuts[j]
        split = get_train_test_split(num_data, "diabetes", cut, i, to_stratify=to_stratify)
        for x in range(tries):
            # wipe the model every time
            new_model = models.clone_model(initial_model)   
            # recompile, train and save
            history, model = model_train(new_model, split, 50) 
            
            model_name = modelDir(i) + get_model_name(cut, params) + "_{}.h5".format(x)
            models.save_model(model, filepath = model_name, include_optimizer = False)
            histories[x].append(history)
            # display_history(history)   

Epoch 1/50
157/157 [==============================] - 2s 6ms/step - loss: 11.7470 - val_loss: 0.5275
Epoch 2/50
157/157 [==============================] - 0s 3ms/step - loss: 9.0331 - val_loss: 0.4987
Epoch 3/50
157/157 [==============================] - 0s 3ms/step - loss: 8.5954 - val_loss: 0.4211
Epoch 4/50
157/157 [==============================] - 0s 3ms/step - loss: 8.2331 - val_loss: 0.3995
Epoch 5/50
157/157 [==============================] - 0s 3ms/step - loss: 7.8743 - val_loss: 0.4616
Epoch 6/50
157/157 [==============================] - 1s 3ms/step - loss: 7.5200 - val_loss: 0.3521
Epoch 7/50
157/157 [==============================] - 0s 3ms/step - loss: 7.2133 - val_loss: 0.4730
Epoch 8/50
157/157 [==============================] - 1s 3ms/step - loss: 6.9552 - val_loss: 0.4070
Epoch 9/50
157/157 [==============================] - 0s 3ms/step - loss: 6.7437 - val_loss: 0.3622
Epoch 10/50
157/157 [==============================] - 0s 3ms/step - loss: 6.5893 - val_loss: 0.285

In [25]:
# import pickle
# import os

# for i, h in enumerate(histories):
#     filename = './histories/model_{}_{}'.format(i)

#     os.makedirs(os.path.dirname(filename), exist_ok=True)
#     with open(filename, "wb") as file:
#         pickle.dump(h, file)

In [27]:
def display_history(history):
    history_dict = history.history
    loss_values = list(map(lambda x : x / 100, history_dict['loss'])) 
    # problematic loss values
    val_loss_values = history_dict['val_loss']
    epochs = range(1, len(loss_values)+1)
    plt.plot(epochs, loss_values, 'bo', label='Training loss')
    plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()